# Correlated gaussian SKL testing

In [1]:
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import numpy as np
import time

In [2]:
tf.__version__

'2.4.1'

In [3]:
dtype=tf.float64

In [4]:
print(dtype)

<dtype: 'float64'>


In [5]:
tf.config.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [6]:
# Utility functions

# return the lower triangle of A in column order i.e. vech(A)
def vech(A):
    count = 0
    c = A.shape[0]
    v = tf.zeros([c * (c + 1) // 2])
    for j in range(c):
        for i in range(j,c):
            v[count] = A[i,j]
            count += 1
    #return th.tensor(v , device=device, dtype=dtype)
    return v

# vech2L   create lower triangular matrix L from vechA
def vech2L(v,n):
    count = 0
    L = tf.Variable(tnp.zeros((n,n), dtype=dtype))
    for j in range(n):
        for i in range(j,n):
            L[i,j].assign(v[count])
            count += count
    return L 

# Batched vech2L input V is nb x n(n+1)/2
def bvech2L(V,nb,n):
    count = 0
    L = tf.Variable(tnp.zeros([nb,n,n], dtype=dtype))
    for j in range(n):
        for i in range(j,n):
            L[...,i,j].assign(V[..., count])
            count += 1
    #return th.tensor(L , device=device, dtype=dtype)
    return L


In [ ]:
vechLk = tf.Variable([  1.00000039208682, 
              0.02548044275764261, 
              0.3525161612610669,
              1.6669144815242515,
              0.9630555318946559,
              1.8382882034659822 ], dtype=dtype);

In [1]:
V=tnp.random.rand(2,6)
L=bvech2L(V,2,3)
L.assign(tnp.add(L, tnp.eye(3)))

NameError: name 'tnp' is not defined

In [9]:
A = tnp.matmul(L, tnp.transpose(L,[0,2,1]))
A

InternalError: Blas xGEMMStridedBatched launch failed : a.shape=[2,3,3], b.shape=[2,3,3], m=3, n=3, k=3, batch_size=2 [Op:BatchMatMulV2]

In [10]:
tf.linalg.cholesky(A)

NameError: name 'A' is not defined

In [36]:
def test_chol(n=4,m=1000):
    nn = int(n*(n+1)//2)
    V=tnp.random.rand(m,nn)
    L=bvech2L(V,m,n)
    L.assign(tnp.add(L, tnp.eye(n))) 
    A = tnp.matmul(L, tnp.transpose(L,[0,2,1]))
    
    start_time = time.time()
    cholA = tf.linalg.cholesky(A)
    runtime = time.time() - start_time

    print(f'batch cholesky took {runtime} seconds')

In [37]:
test_chol(10,10000)

batch cholesky took 0.0010144710540771484 seconds


In [109]:
def b_energyrc(x,n,nb,Mass,Qmat,Sym,symc):
    
    nx = tf.size(x);
    nn = int(n*(n+1)/2);
    nsym = len(symc);
    
    # extract linear coefs "eigen vector"
    c = x[-nb:];
    # reshape non-linear variables for easier indexing
    X = tnp.reshape(x[:nb*nn], (nb,nn))
    
    # generate tensor of lower triangular matrices from X
    # these are the non-linear parameters of the basis set
    L = tf.zeros((nb,n,n),dtype=dtype)
    L = bvech2L(X,nb,n)
    #return L
    # get the determinates for L |L| is the product of diag elements
    detL = tf.abs(tf.math.reduce_prod(tf.linalg.diag_part(L),1))

    # create the tensor of matrix products of the L matrices AKL = L x Ltranspose
    AK = tf.matmul(L,tf.transpose(L, perm=[0, 2, 1]))
    
    # Initialize H T V and S matrices
    # H = T + V, we are solving (H-ES)c = 0 for E (energy)
    H = tf.zeros((nb,nb), dtype=dtype);
    S = tf.zeros((nb,nb), dtype=dtype);
    T = tf.zeros((nb,nb), dtype=dtype);
    V = tf.zeros((nb,nb), dtype=dtype);
    

    # outer loop is over symmetry terms, the matrices are summed over these sym terms
    #for k in range(0,nsym):
    for k in range(0,1):
        
        P = Sym[k,:,:]
        # symetry projection is applied only to "ket" this constructs AL
        AL = tf.matmul(tf.transpose(P), tf.matmul(AK,P))

        # Akl = Ak + Al
        AKL = tf.zeros((nb,nb,n,n), dtype=dtype)
        #for i in range(nb):
        #    for j in range(nb):
        #        #AKL[i,j] =  th.add(AK[i], AL[j])
        #        AKL[i,j] =  AK[i] + AL[j]
        AKL = tf.repeat(AL, repeats=[nb,1,1,1]) + tf.transpose(tf.repeat(AK, repeats=[1,nb,1,1]))
        
        return AKL
    
        # get the Cholesky decomp of all Akl martices
        cholAKL = cholesky(AKL)
        
        # get determinates of AKL from diags |Akl|= |Lk|**2
        detAKL = th.prod(th.diagonal(cholAKL, offset=0, dim1=-1, dim2=-2),-1)**2
        
        # compute inverses of lower tringular matrices in cholAKL
        invLKL = inverseL(cholAKL)
        
        # inverses Akl^-1 = Lkl' x Lkl
        invAKL = th.matmul(th.transpose(invLKL, dim0=-1, dim1=-2),invLKL)

        # get terms needed for potential energy V
        RIJ = th.zeros_like(invAKL, device=device, dtype=dtype);
        # 1/rij i~=j
        for j in range(0,n-1):
            for i in range(j+1,n):
                tmp2 = invAKL[...,i,i] + invAKL[...,j,j] - 2*invAKL[...,i,j];
                RIJ[...,i,j] = th.rsqrt(tmp2)

        # 1/rij i=j
        for i in range(0,n):
            RIJ[...,i,i] = th.rsqrt(invAKL[...,i,i])    

        # MATRIX ELEMENTS
        
        # Overlap: (normalized)
        # Skl = 2^3n/2 (||Lk|| ||Ll||/|AKL|)^3/2
        SKL = 2**(n*1.5) * th.sqrt( th.pow(th.ger(detL, detL)/detAKL ,3) );

        # Kinetic energy
        #TKL = SKL*(6*th.trace(Mass@Ak@invAkl@Al)) = skl*(6*th.sum(Mass*(Ak@invAkl@Al)))

        Tmat = th.zeros_like(invAKL)
        #for i in range(nb):
        #    for j in range(nb):
        #        Tmat[i,j] = (AK[i]@invAKL[i,j]@AL[j])
        Tmat = th.matmul(th.transpose(AK.repeat((nb,1,1,1)), 0,1), th.matmul(invAKL,AL))
        TKL = 6*SKL*th.sum(Mass*Tmat, dim=(-2,-1))

        # potential energy
        TWOoSqrtPI = 1.1283791670955126 # 2/sqrt(pi)
        
        VKL = TWOoSqrtPI*SKL*th.sum(RIJ*Qmat, dim=(-2,-1))
    
        # accumulate matrices over sym terms
        S = S + symc[k]*SKL
        T = T + symc[k]*TKL
        V = V + symc[k]*VKL
        
    # Hamiltonian
    H = T + V
    
    # complete lower triangle of H and S
    #for i in range(0,nb):
    #    for j in range(i+1,nb):
    #        H[j,i] = H[i,j]
    #        S[j,i] = S[i,j]
    #        #H[i,j] = H[j,i];
    #        #S[i,j] = S[j,i];
    H = th.triu(H,1)+th.t(th.triu(H))
    S = th.triu(S,1)+th.t(th.triu(S))
    # compute Rayleigh quotent (it is the smallest energy eigen value when minimized over c)
    cHc = c@H@c;
    cSc = c@S@c;
    eng = cHc/cSc;
    
    return eng           

In [110]:
def opt_energyrc(steps=1, num_basis=8, restart=True):
    
    #
    # Li BO setup
    #
    n=3
    
    Mass = tf.constant([[0.5, 0.0, 0.0],
                     [0.0, 0.5, 0.0],
                     [0.0, 0.0, 0.5]], dtype=dtype);
    
    Chg = tf.constant([-3, 1, 1, -3, 1, -3], dtype=dtype);
    Charge = vech2L(Chg,n)
    
    # symmetry projection terms
    Sym = tf.Variable(tnp.zeros((6,3,3), dtype=dtype))
    # (1)(2)(3)
    Sym[0,:,:].assign(tf.constant([[1,0,0],[0,1,0],[0,0,1]], dtype=dtype));
    # (12)
    Sym[1,:,:].assign(tf.constant([[0,1,0],[1,0,0],[0,0,1]], dtype=dtype));
    # (13)
    Sym[2,:,:].assign(tf.constant([[0,0,1],[0,1,0],[1,0,0]], dtype=dtype));
    # (23)
    Sym[3,:,:].assign(tf.constant([[1,0,0],[0,0,1],[0,1,0]], dtype=dtype));
    # (123)
    Sym[4,:,:].assign(tf.constant([[0,1,0],[0,0,1],[1,0,0]], dtype=dtype));
    # (132)
    Sym[5,:,:].assign(tf.constant([[0,0,1],[1,0,0],[0,1,0]], dtype=dtype));

    # coeff's
    symc = tf.constant([4.0,4.0,-2.0,-2.0,-2.0,-2.0], dtype=dtype);
    #symc = th.tensor([4.0,4.0,-2.0], device=device, dtype=dtype);
    
    # Sample parameters should return energy of -7.3615
    xvechL=tf.Variable([
         1.6210e+00, -2.1504e-01,  9.0755e-01,  9.7866e-01, -2.8418e-01,
        -3.5286e+00, -3.3045e+00, -4.5036e+00, -3.2116e-01, -7.1901e-02,
         1.5167e+00, -8.4489e-01, -2.1377e-01, -3.6127e-03, -5.3774e-03,
        -2.1263e+00, -2.5191e-01,  2.1235e+00, -2.1396e-01, -1.4084e-03,
        -1.0092e-02,  4.5349e+00,  9.4837e-03,  1.1225e+00, -2.1315e-01,
         5.8451e-02, -4.9410e-03,  5.0853e+00,  7.3332e-01,  5.0672e+00,
        -2.1589e-01, -6.8986e-03, -1.4310e-02,  1.5979e+00,  3.3946e-02,
        -8.7965e-01, -1.1121e+00, -2.1903e-03, -4.6925e-02,  2.1457e-01,
         3.3045e-03,  4.5120e+00, -2.1423e-01, -1.6493e-02, -2.3429e-03,
        -8.6715e-01, -6.7070e-02,  1.5998e+00
     ], dtype=dtype)

    evec = tf.Variable([
      -6.0460e-02,  7.7708e-05, 1.6152e+00,  9.5443e-01,  
      1.1771e-01,  3.2196e+00,  9.6344e-01, 3.1398e+00
    ], dtype=dtype)

    
    # uncomment following lines to test above 
    nb=8
    x1 = tf.Variable(tnp.concatenate((xvechL,evec)), dtype=dtype)
    energy = b_energyrc(x1,n,nb,Mass,Charge,Sym,symc) 
    #print(energy) # should be -7.3615
    #return x1
    return energy
    
    if restart:
        nb=num_basis
        x1 = xrestart
    else:
        # random start point
        nb=num_basis
        #th.manual_seed(333)
        x1 = th.empty(int(nb*n*(n+1)/2 + nb), device=device, dtype=dtype, requires_grad=True)
        th.nn.init.uniform_(x1, a=-0.8, b=0.8)
        
    # start from a restart value
    #x1 = xrestart
    #print(energy)
    #return x1
    
    # Do the Optimization
    #optimizer = th.optim.LBFGS([x1])
    #optimizer = th.optim.Adadelta([x1], lr=160.0)
    #optimizer = th.optim.Adam([x1], lr=0.00005)
    optimizer = th.optim.Rprop([x1], lr=0.0001, etas=(0.5, 1.2), step_sizes=(1e-07, 50))
    
    #scheduler = th.optim.lr_scheduler.ReduceLROnPlateau(optimizer,threshold=0.00001,cooldown=3, verbose=True,patience=2, factor=0.5)
    
    for i in range(steps):
        optimizer.zero_grad()
        loss = b_energyrc(x1,n,nb,Mass,Charge,Sym,symc)
        loss.backward()
        #def closure():
        #    return b_energyrc(x1,n,nb,Mass,Charge,Sym,symc)
        #optimizer.step(closure)
        optimizer.step()
        #scheduler.step(loss)
        
        if (i<20 or not i%100):print('step: {:5}  f: {:4.12f}  gradNorm: {:.9f}'.format(i, loss, th.norm(x1.grad)))
    # print last value
    print('step: {:5}  f: {:4.12f}  gradNorm: {:.9f}'.format(i, loss, th.norm(x1.grad)))
    return x1

In [111]:
opt_energyrc()

ValueError: Dimensions 72 and 4 are not compatible